<a href="https://colab.research.google.com/github/sushantagarwal29/ragpoc/blob/main/CaseStudy3-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pandas
%pip install llama-index
%pip install kdbai-client
%pip install llama-index-core
%pip install --upgrade openai
%pip install -U llama-index-llms-azure-inference
%pip install -U llama-index-embeddings-azure-inference
%pip install llama-index-embeddings-openai
%pip install llama-parse
%pip install llama-index-vector-stores-kdbai

In [ ]:
import os
from getpass import getpass
# Set up environment variables for API keys and endpoints
os.environ["LLAMA_CLOUD_API_KEY"] = (
    os.environ["LLAMA_CLOUD_API_KEY"]
    if "LLAMA_CLOUD_API_KEY" in os.environ
    else getpass("LLAMA CLOUD API key: ")
)

AZURE_OPENAI_ENDPOINT = (
    os.environ["AZURE_OPENAI_ENDPOINT"]
    if "AZURE_OPENAI_ENDPOINT" in os.environ
    else input("AZURE_OPENAI_ENDPOINT: ")
)
AZURE_OPENAI_API_KEY = (
    os.environ["AZURE_OPENAI_API_KEY"]
    if "AZURE_OPENAI_API_KEY" in os.environ
    else getpass("AZURE_OPENAI_API_KEY: ")
)

# Initialize the LLM and embedding models
llm = AzureAICompletionsModel(
    endpoint=AZURE_OPENAI_ENDPOINT + "/openai/deployments/gpt-4o",
    credential=AZURE_OPENAI_API_KEY,
    api_version="2024-08-01-preview",
)

embed_model = AzureAIEmbeddingsModel(
    endpoint=AZURE_OPENAI_ENDPOINT + "/openai/deployments/text-embedding-3-small",
    credential=AZURE_OPENAI_API_KEY,
    model_name="text-embedding-3-small",
)

# Set the LLM and embedding model in the Settings object
Settings.llm = llm
Settings.embed_model = embed_model

AZURE_OPENAI_ENDPOINT: https://ai-depoc1aihub1643128651037.openai.azure.com/
AZURE_OPENAI_API_KEY: ··········


In [ ]:
import os
import pandas as pd
from getpass import getpass
from llama_index.llms.azure_inference import AzureAICompletionsModel
from llama_index.embeddings.azure_inference import AzureAIEmbeddingsModel
from llama_parse import LlamaParse
from llama_index.core import Settings
from openai import AzureOpenAI


#client = OpenAI()
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key=AZURE_OPENAI_API_KEY,
  api_version="2024-08-01-preview"
)

# Define a function to convert query into its embeddings to enable semantic search
def embed_query(query):
    query_embedding = client.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )
    return query_embedding.data[0].embedding

# Define a function to retrieve relevant chunks based on query
def retrieve_data(query):
    query_embedding = embed_query(query)
    results = table.search(vectors={'flat':[query_embedding]},n=5)
    print(results)

    retrieved_data_for_RAG = []
    for index, row in results[0].iterrows():
      retrieved_data_for_RAG.append(row['text'])
    return retrieved_data_for_RAG


# Define a function to call the LLM with prompt along with relevant context and get structured output
def RAG(query):
    question = "You will answer this question based on the provided reference material: " + query
    messages = "Here is the provided context: " + "\n"
    results = retrieve_data(query)
    print(results)




In [ ]:
import os
import pandas as pd
from getpass import getpass
from llama_index.llms.azure_inference import AzureAICompletionsModel
from llama_index.embeddings.azure_inference import AzureAIEmbeddingsModel
from llama_parse import LlamaParse
from llama_index.core import Settings
from openai import AzureOpenAI


#client = OpenAI()
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key=AZURE_OPENAI_API_KEY,
  api_version="2024-08-01-preview"
)

# Define a function to convert query into its embeddings to enable semantic search
def embed_query(query):
    query_embedding = client.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )
    return query_embedding.data[0].embedding

# Define a function to retrieve relevant chunks based on query
def retrieve_data(query):
    query_embedding = embed_query(query)
    results = table.search(vectors={'flat':[query_embedding]},n=5)
    retrieved_data_for_RAG = []
    for index, row in results[0].iterrows():
      retrieved_data_for_RAG.append(row['text'])
    return retrieved_data_for_RAG


# Define a function to call the LLM with prompt along with relevant context and get structured output
def RAG(query):
    question = "You will answer this question based on the provided reference material: " + query
    messages = "Here is the provided context: " + "\n"
    results = retrieve_data(query)
    if results:
      for data in results:
        messages += data + "\n"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": question},
            {
          "role": "user",
          "content": [
              {"type": "text", "text": messages},
          ],
          }
        ],
         response_format=
     {
      "type": "json_schema",
      "json_schema": {
        "name": "Daily_Sports_Highlights",
        "schema": {
          "type": "object",
          "properties": {
            "date": { "type": "string" },
            "docName": { "type": "string" },
            "sectionHeading": { "type": "string" },
            "articleTitle": { "type": "string" },
            "source": { "type": "string" },
            "articleDate": { "type": "string" },
            "link": { "type": "string" },
            "content": { "type": "string" }
          },
        },
      }
    },
        max_tokens=300,
    )
    content = response.choices[0].message.content
    return content




Functions for extracting unique sports sections

In [ ]:
import os
import pandas as pd
from getpass import getpass
from llama_index.llms.azure_inference import AzureAICompletionsModel
from llama_index.embeddings.azure_inference import AzureAIEmbeddingsModel
from llama_parse import LlamaParse
from llama_index.core import Settings
from openai import AzureOpenAI


#client = OpenAI()
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_key=AZURE_OPENAI_API_KEY,
  api_version="2024-08-01-preview"
)

# Define a function to convert query into its embeddings to enable semantic search
def embed_query(query):
    query_embedding = client.embeddings.create(
            input=query,
            model="text-embedding-3-small"
        )
    return query_embedding.data[0].embedding

# Define a function to retrieve relevant chunks based on query
def retrieve_data(query):
    query_embedding = embed_query(query)
    results = table.search(vectors={'flat':[query_embedding]},n=5)
    retrieved_data_for_RAG = []
    for index, row in results[0].iterrows():
      retrieved_data_for_RAG.append(row['text'])
      print(row['text'])
    return retrieved_data_for_RAG


# Define a function to call the LLM with prompt along with relevant context and get structured output
def RAG(query):
    question = "You will answer this question based on the provided reference material: " + query
    messages = "Here is the provided context: " + "\n"
    results = retrieve_data(query)
    if results:
      for data in results:
        messages += data + "\n"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": question},
            {
          "role": "user",
          "content": [
              {"type": "text", "text": messages},
          ],
          }
        ],
         response_format=
     {
      "type": "json_schema",
      "json_schema": {
        "name": "Daily_Sports_Highlights",
        "schema": {
          "type": "object",
          "properties": {
            "sectionHeading": { "type": "string" }
          },
        },
      }
    },
        max_tokens=300,
    )
    content = response.choices[0].message.content
    return content




In [ ]:
# Get the current working directory and set up folders
current_dir = os.getcwd()
input_folder = os.path.join(current_dir, 'input')
output_folder = os.path.join(current_dir, 'output')
processed_folder = os.path.join(current_dir, 'processed')

os.makedirs(output_folder, exist_ok=True)
os.makedirs(input_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)

In [ ]:
import nest_asyncio

# Apply nest_asyncio to allow running async code in a notebook
nest_asyncio.apply()
import kdbai_client as kdbai
#Set up KDB.AI endpoint and API key
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

#connect to KDB.AI
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

schema = [
        dict(name="document_id", type="str"),
        dict(name="text", type="str"),
        dict(name="embeddings", type="float32s"),
    ]

indexFlat = {
        "name": "flat",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 1536, 'metric': 'L2'},
    }

# Connect with kdbai database
db = session.database("default")


KDBAI_TABLE_NAME = "LlamaParse_Table"

# First ensure the table does not already exist
try:
    db.table(KDBAI_TABLE_NAME).drop()
except kdbai.KDBAIException:
    pass

#Create the table
table = db.create_table(KDBAI_TABLE_NAME, schema, indexes=[indexFlat])

KDB.AI endpoint: https://cloud.kdb.ai/instance/5mpjbggrkg
KDB.AI API key: ··········


In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["/content/input/Daily_Sports_Highlights_01092024_With_Subsections.pdf"]).load_data()

In [ ]:
print(documents[0].text[:10000])

01.09.2024
Daily Sports Highlights
Football
Lionel Messi Leads Inter Miami to Victory in MLS Cup Final
Source: ESPN - September 1, 2024
Lionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final.
Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one
of the greatest footballers in the history of the sport.
Link: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final
Cristiano Ronaldo Scores 800th Career Goal
Source: Sky Sports - September 1, 2024
Cristiano Ronaldo has reached another milestone in his illustrious career by scoring his 800th career goal. The
Portuguese star continues to break records and is widely regarded as one of the greatest footballers of all time.
Link: https://www.skysports.com/football/ronaldo-scores-800th-goal
Tennis
Serena Williams Announces Retirement After US Open
Source: BBC Sports - September 1, 202

In [ ]:
from llama_index.core.node_parser import (
    SemanticSplitterNodeParser,
)

splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)


In [ ]:
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
print(nodes[1].get_content())

This victory cements Messi's status as one
of the greatest footballers in the history of the sport.
Link: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final
Cristiano Ronaldo Scores 800th Career Goal
Source: Sky Sports - September 1, 2024
Cristiano Ronaldo has reached another milestone in his illustrious career by scoring his 800th career goal. The
Portuguese star continues to break records and is widely regarded as one of the greatest footballers of all time.
Link: https://www.skysports.com/football/ronaldo-scores-800th-goal
Tennis
Serena Williams Announces Retirement After US Open
Source: BBC Sports - September 1, 2024
Tennis legend Serena Williams has announced that she will retire from professional tennis after the US Open. Williams,
who has won 23 Grand Slam singles titles, is considered one of the greatest tennis players of all time.
Link: https://www.bbc.com/sport/tennis/serena-williams-retires
Rafael Nadal Announces Full Recovery, Set to Comp

In [ ]:
import os
import pandas as pd
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.kdbai import KDBAIVectorStore

pdf_file_path = "/content/input/Daily_Sports_Highlights_01092024_With_Subsections.pdf"

# Load and parse the PDF
parsing_instructions = '''The document contains higlights of multiple sports news. Each section corresponds to a particular sports. Extract all articles section wise.'''

documents = LlamaParse(result_type="markdown",parsing_instructions=parsing_instructions).load_data(pdf_file_path)

Started parsing the file under job_id 41209774-3940-40ba-b732-eba53bd3bdc1


In [ ]:


#print(documents[0].text[:1000])

        # Parse the documents using MarkdownElementNodeParser
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8).from_defaults()
        #print(node_parser)

nodes = node_parser.get_nodes_from_documents(documents)
        #print(nodes)
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

        # First ensure the table does not already exist
try:
  db.table(KDBAI_TABLE_NAME).drop()
except kdbai.KDBAIException:
  pass

        #Create the table
table = db.create_table(KDBAI_TABLE_NAME, schema, indexes=[indexFlat])
        ## vectore store
vector_store = KDBAIVectorStore(table)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

        #Create the index, inserts base_nodes and objects into KDB.AI
recursive_index = VectorStoreIndex(nodes= base_nodes, storage_context=storage_context
        #,store_nodes_override=FALSE
        )

table.query()

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


,document_id,text,embeddings
0,f63286bc-0d16-412b-82a4-25604680d96f,01.09.2024\n\n Daily Sports Highlights\n\n Foo...,"[-0.004217085, 0.005169037, 0.033469893, 0.024..."
1,512f9d2b-ec4e-49d3-812c-264532213803,Sports News\n\n Tennis\n\nRafael Nadal has ann...,"[0.018013053, 0.032876156, 0.03485946, 0.02374..."
2,dbfb1c44-880d-414a-bfb8-06950ea6d542,Max Verstappen Wins Italian Grand Prix\n\nSour...,"[-0.062151738, 0.00426373, -0.00141799, 0.0334..."


In [ ]:
import os
import pandas as pd
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.kdbai import KDBAIVectorStore

pdf_file_path = "/content/input/Daily_Sports_Highlights_01092024_With_Subsections.pdf"

# Load and parse the PDF
parsing_instructions = '''The document contains higlights of multiple sports news. Each section corresponds to a particular sports. Extract all articles section wise.'''

documents = LlamaParse(result_type="markdown",parsing_instructions=parsing_instructions).load_data(pdf_file_path)

#print(documents[0].text[:1000])

        # Parse the documents using MarkdownElementNodeParser
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8).from_defaults()
        #print(node_parser)

nodes = node_parser.get_nodes_from_documents(documents)
        #print(nodes)
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

        # First ensure the table does not already exist
try:
  db.table(KDBAI_TABLE_NAME).drop()
except kdbai.KDBAIException:
  pass

        #Create the table
table = db.create_table(KDBAI_TABLE_NAME, schema, indexes=[indexFlat])
        ## vectore store
vector_store = KDBAIVectorStore(table)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

        #Create the index, inserts base_nodes and objects into KDB.AI
recursive_index = VectorStoreIndex(nodes= base_nodes, storage_context=storage_context
        #,store_nodes_override=FALSE
        )

table.query()

Started parsing the file under job_id ca174345-d21a-4196-9694-78de8c2d5437


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


,document_id,text,embeddings
0,d88f5650-f0d0-42fe-914f-274dace3ba44,01.09.2024\n\n Daily Sports Highlights\n\n Foo...,"[-0.004217085, 0.005169037, 0.033469893, 0.024..."
1,51a29827-5bf8-4073-9c2b-e0e5a93843ae,Sports News\n\n Tennis\n\nRafael Nadal has ann...,"[0.018013053, 0.032876156, 0.03485946, 0.02374..."
2,27e3e034-8bad-411c-9979-f6f0f42ec159,Max Verstappen Wins Italian Grand Prix\n\nSour...,"[-0.062151738, 0.00426373, -0.00141799, 0.0334..."


In [ ]:


        # # Extract text from nodes
        # extracted_texts = [node.text for node in nodes]

        # # Process only the first batch of combined text
        # batch_size = 5  # Adjust batch size as needed
        # first_batch = next(batch_process_texts(extracted_texts, batch_size))  ## take first batch only to store in csv

        query = """You are an AI assistant specialized in analyzing sports news.
        Your task is to extract relevant information for each article in a given document related to Golf only.
        Your output must be a structured JSON object.

        Instructions:
        1. Carefully read the entire documents
        2. Extract each article seperately
        3. Present your findings in JSON format as specified below.

        Expected JSON keys and explanation of what they are:
        - 'date': The date of the sports article in mm/dd/yyyy format.
        - 'docName': Name of the document.
        - 'sectionHeading': Heading of the section.
        - 'articleTitle': Title of the article.
        - 'source': Source of the article.
        - 'articleDate': Date of the article in mm/dd/yyyy format.
        - 'link': Link to the article.
        - 'content': Content of the article.
        """

        # Get RAG output for the query
        # rag_output = RAG(query, first_batch)
        rag_output = RAG(query)
        print(rag_output)

{"date":"09/01/2024","docName":"Daily Sports Highlights","sectionHeading":"Golf","articleTitle":"Tiger Woods Inducted into World Golf Hall of Fame","source":"Golf Digest","articleDate":"09/01/2024","link":"","content":"Tiger Woods has been officially inducted into the World Golf Hall of Fame. Woods, one of the most successful golfers in history, has had a profound impact on the sport and continues to inspire future generations."}


extract only sections

In [ ]:


        # # Extract text from nodes
        # extracted_texts = [node.text for node in nodes]

        # # Process only the first batch of combined text
        # batch_size = 5  # Adjust batch size as needed
        # first_batch = next(batch_process_texts(extracted_texts, batch_size))  ## take first batch only to store in csv

        query = """You are an AI assistant specialized in analyzing sports news.
        Your task is to extract all sports which have news articles available in the document.
        Your output must be a structured JSON object.

        Instructions:
        1. Carefully read the entire documents
        2. Extract each article seperately
        3. Present your findings in JSON format as specified below.

        Expected JSON keys and explanation of what they are:
        - 'sectionHeading': Heading of the section.
        """

        # Get RAG output for the query
        # rag_output = RAG(query, first_batch)
        rag_output = RAG(query)
        print(rag_output)

01.09.2024

 Daily Sports Highlights

 Football

 Lionel Messi Leads Inter Miami to Victory in MLS Cup Final

Source: ESPN - September 1, 2024

Lionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final. Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one of the greatest footballers in the history of the sport.

Link: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final

 Cristiano Ronaldo Scores 800th Career Goal

Source: Sky Sports - September 1, 2024

Cristiano Ronaldo has reached another milestone in his illustrious career by scoring his 800th career goal. The Portuguese star continues to break records and is widely regarded as one of the greatest footballers of all time.

Link: https://www.skysports.com/football/ronaldo-scores-800th-goal

 Tennis

 Serena Williams Announces Retirement After US Open

Source: BBC Sports

In [ ]:
############# final code with nodes  ##################
import json
import os
import pandas as pd
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.kdbai import KDBAIVectorStore

# def batch_process_texts(texts, batch_size):
#     for i in range(0, len(texts), batch_size):
#         yield " ".join(texts[i:i + batch_size])

# Iterate over all PDFs in the input folder and process them
for pdf_file_name in os.listdir(input_folder):
    if pdf_file_name.endswith('.pdf'):
        pdf_file_path = os.path.join(input_folder, pdf_file_name)

        # Load and parse the PDF
        parsing_instructions = '''The document contains higlights of multiple sports news. Each section corresponds to a particular sports
        '''
        documents = LlamaParse(result_type="markdown", parsing_instructions=parsing_instructions).load_data(pdf_file_path)
        print(documents[0].text[:1000])

        # Parse the documents using MarkdownElementNodeParser
        node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8).from_defaults()
        #print(node_parser)

        nodes = node_parser.get_nodes_from_documents(documents)
        #print(nodes)
        base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

        # First ensure the table does not already exist
        try:
            db.table(KDBAI_TABLE_NAME).drop()
        except kdbai.KDBAIException:
          pass

        #Create the table
        table = db.create_table(KDBAI_TABLE_NAME, schema, indexes=[indexFlat])
        ## vectore store
        vector_store = KDBAIVectorStore(table)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        #Create the index, inserts base_nodes and objects into KDB.AI
        recursive_index = VectorStoreIndex(
        nodes= base_nodes, storage_context=storage_context
        #,store_nodes_override=FALSE
        )
        table.query()

        # # Extract text from nodes
        # extracted_texts = [node.text for node in nodes]

        # # Process only the first batch of combined text
        # batch_size = 5  # Adjust batch size as needed
        # first_batch = next(batch_process_texts(extracted_texts, batch_size))  ## take first batch only to store in csv

        query = """You are an AI assistant specialized in analyzing sports news.
        Your task is to extract relevant information for each article in a given document
        Your output must be a structured JSON object.

        Instructions:
        1. Carefully read the entire documents
        2. Extract each article seperately
        3. Present your findings in JSON format as specified below.

        Expected JSON keys and explanation of what they are:
        - 'date': The date of the sports article in mm/dd/yyyy format.
        - 'docName': Name of the document.
        - 'sectionHeading': Heading of the section.
        - 'articleTitle': Title of the article.
        - 'source': Source of the article.
        - 'articleDate': Date of the article in mm/dd/yyyy format.
        - 'link': Link to the article.
        - 'content': Content of the article.
        """

        # Get RAG output for the query
        # rag_output = RAG(query, first_batch)
        rag_output = RAG(query)
        print(rag_output)

        # Convert the result to text form
        if rag_output:
            try:
                json_file_name = f"{os.path.splitext(pdf_file_name)[0]}.pdf"
                json_path = os.path.join(output_folder, json_file_name)
                # Write JSON to file
                with open(json_path, "w") as json_file:
                  json.dump(rag_output, json_file, indent=4)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON: {rag_output}")

        # Move processed PDF to processed folder
        #processed_pdf_path = os.path.join(processed_folder, pdf_file_name)
        #os.rename(pdf_file_path, processed_pdf_path)

        #table.drop()
print("Processing complete.")

Started parsing the file under job_id 8c97c858-11b1-4515-bab5-77d24c535d14
.# 01.09.2024

# Daily Sports Highlights

# Football

# Lionel Messi Leads Inter Miami to Victory in MLS Cup Final

Source: ESPN - September 1, 2024

Lionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final. Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one of the greatest footballers in the history of the sport.

Link: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final

# Cristiano Ronaldo Scores 800th Career Goal

Source: Sky Sports - September 1, 2024

Cristiano Ronaldo has reached another milestone in his illustrious career by scoring his 800th career goal. The Portuguese star continues to break records and is widely regarded as one of the greatest footballers of all time.

Link: https://www.skysports.com/football/ronaldo-scores-800th-goal

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


{"date":"01/09/2024","docName":"Daily Sports Highlights","sectionHeading":"Football","articleTitle":"Lionel Messi Leads Inter Miami to Victory in MLS Cup Final","source":"ESPN","articleDate":"09/01/2024","link":"https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final","content":"Lionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final. Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one of the greatest footballers in the history of the sport."}
Processing complete.


In [ ]:
print(nodes)

[TextNode(id_='db03886d-e45c-4ccb-a3ab-656e1f8aabdb', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6c5407fc-4fef-4821-8696-f7d7f1eb695e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='a1141e5e501a56ca9ad9fe549cb06eca40aaeb5e21164d12d7c99df6d046f74f')}, metadata_template='{key}: {value}', metadata_separator='\n', text="01.09.2024\n\n Daily Sports Highlights\n\n Football\n\n Lionel Messi Leads Inter Miami to Victory in MLS Cup Final\n\nSource: ESPN - September 1, 2024\n\nLionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final. Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one of the greatest footballers in the history of the sport.\n\nLink: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-fina